<a href="https://colab.research.google.com/github/kslim1025/TF_Keras_RNN/blob/master/RNN_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MNIST 문자열 분석으로 유명한 데이터셋이 패션에 관한 데이터 세트를 만듬 그게 FashionMINIST 데이터셋이다.
# 데이터 이미지가 0에서 255까지 값을 가지는 28x28이미지라는 것을 확인가능
# 정답이 되는 라벨을 확인하기 위해 print를 붙여서  확인
# 외부 데이터를 이용한 정제과정
# ctrl+enter를 사용한 런타임가능
# !nvidia-smi : 어떤 GPU를 사용하는지 확인가능한 명령어 
#

try:
  # %tensorflow_version only exists in Colab
  %tensorflow_version 2.x
except Exception:
  pass

# 넘파이는 수학과 과학 연산에 특화된 파이썬 모듈로 딥러닝에서도 유용하게 사용된다.

import numpy as np;
import tensorflow as tf;
import pandas as pd;
import matplotlib.pyplot as plt;
import math;
################################################################################
!nvidia-smi
       
#+-----------------------------------------------------------------------------+
#| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
#|-------------------------------+----------------------+----------------------+
#| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
#| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
#|                               |                      |               MIG M. |
#|===============================+======================+======================|
#|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
#| N/A   51C    P0    35W / 250W |   1581MiB / 16280MiB |      0%      Default |
#|                               |                      |                 ERR! |
#+-------------------------------+----------------------+----------------------+
#                                                                               
#+-----------------------------------------------------------------------------+
#| Processes:                                                                  |
#|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
#|        ID   ID                                                   Usage      |
#|=============================================================================|
#|  No running processes found                                                 |
#+-----------------------------------------------------------------------------+

################################################################################

X = []
Y = []

for i in range(3000):
  # 0 ~ 1 범위의 랜덤한 숫자 100개를 만듭니다.
  lst = np.random.rand(100)
  
  # 마킹할 숫자 2개의 인덱스를 뽑습니다.
  idx = np.random.choice(100, 2, replace=False)
  # 마킹 인덱스가 저장된 원-핫 인코딩 벡터를 만듭니다
  zeros = np.zeros(100)
  zeros[idx] = 1

  # 마킹 인덱스와 랜덤한 숫자를 합쳐서 X에 저장합니다.
  X.append(np.array(list(zip(zeros, lst))))
  #마킹 인덱스가 1인 값만 서로 곱해서 Y에 저장합니다
  Y.append(np.prod(lst[idx]))

print(X[0], Y[0])

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()


model = tf.keras.Sequential([
                             tf.keras.layers.GRU(units=30, return_sequences=True, input_shape=[100,2]),
                             tf.keras.layers.GRU(units=30),
                             tf.keras.layers.Dense(1)
])

model.compile(optimizer= 'adam', loss='mse')
model.summary()

X = np.array(X)
Y = np.array(Y)

#GRU 레이러를 사용한 네트워크 파라미터 수는 LSTM레이어 네트워크 보다 약23.3% 감소한 수치입니다.
#2560개의 데이터만 학습 진행, 검증 데이터는 20%로 지정
history = model.fit(X[:2560], Y[:2560], epochs=100, validation_split=0.2)

plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

model.evaluate(X[2560:], Y[2560:])
prediction = model.predict(X[:2560:2560+5])
# 5개의 테스트 데이터에 대한 예측을 표시하기
for i in range(5):
  print(Y[2560+i], '\t', prediction[i][0], '\tdiff:', abs(prediction[i][0] - Y[2560+i]))
  prediction = model.predict(X[2560:])
  cnt = 0
  for i in range(len(prediction)):
    #오차가 0.04 이상이면 오답입니다.
    if abs(prediction[i][0] - Y[2560+i])> 0.04:
      cnt += 1
#정확도가 97% 가까이나옴
print('correctness:', (440 - cnt) / 440 * 100, '%')

# Embedding Layer(임베딩 레이어)
# 자연어를 수치화된 정보로 바꾸기 위한 레이어입니다.

# 자연어는 시간의 흐름에 따라 정보가 연속적으로 이어지는 시퀀스 데이터입니다.
# 이미지를 픽셀 단위로 잘게 쪼갤 수 있듯이 자연어도 정보를 잘게 쪼갤 수 잇습니다. 영어는 문자(character), 한글은 문자를 넘어 자소 단위로도 쪼갤 수 있습니다.
# 이보다 더 큰 단위로는 띄어쓰기 단위인 단어가 있습니다. 또 이 방법들과 다르게 몇개의 문자를 묶어서 파악하려는 n-gram 기법입니다. 예를 들어'this is it'
# 이라는 문장을 3개의 문자를 묶은 3-gram으로 나타내려면 ['thi','his','is ','s i','is','is ','s i', ' it', 'it.']이라는 배열로 나타낼 수 있습니다.
# 딥러닝 기법의 발달로 이후는 n-gram보다 단어나 문자 단위의 자연어 처리가 많이 쓰이고 있습니다.

# 즉 Word Embedding > n-gram보다 좋은 성능을 냅니다.
# 임베딩 레이어보다 좀 더 쉬운 기법은 자연어를 구성하는 단위에 대해 정수를 (index)를 저장하는 방법입니다.
#단어를 기반으로 정 수 인덱스를 저장하는 예를 들면 'this is a big cat'이라는 문장에 대해 정수 인덱스를 저장하면 처음 나오는 단어부터 인덱스를 저장합니다.

#임베딩 레이어에 대한 개념은 쉬운 편이지만 학습시키는 방법에는 여러가지가 있습니다. 대표적인 방법은 Word2Vec, GloVe, FastText, ELMo등이 있습니다.
#이 방법론으로 미리 훈련된 임베딩 레이어의 가중치를 불러와서 사용하면 시간을 절약할 수 있습니다.





# 긍정, 부정 감성 분석
# 감성분석은 입력된 자연어 안의 주관적의견, 감정 등을 찾아낸다.
# 이 가운데 극성(polarity) 감성 분석은 문장의 궁정/부정이나 긍정/중립/부정으로 분류합니다.
#영화 리뷰이나 음식점 리뷰는 데이터의 양이 많고 별점을 함꼐 달기 때문에 긍정/중립/부정 라벨링이 쉬워서 극성 감성 분석에 쉽게 적용할 수 있습니다.


Thu Oct  1 08:54:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

NameError: ignored